In [1]:
import numpy as np
from scipy import signal as sc
import matplotlib.pyplot as plt
from __future__ import division
%matplotlib

Using matplotlib backend: Qt5Agg


In [33]:
# Filter design functions
def makeLowShelf (fc, Gain, fs):
    b = np.zeros (3)
    a = np.zeros (3)

    K = np.tan(np.pi * fc / fs)#bilinear transform frequency correction

    if Gain >= 0:
        V = 10.**(Gain / 20.)
        Den = (1. + np.sqrt(2.) * K + K**2)
        
        b[0] = (1. + np.sqrt(2.*V) * K + V*K**2) / Den
        b[1] = 2. * (V*K**2 - 1) / Den
        b[2] = (1 - np.sqrt(2. * V) * K + V*K**2) / Den
        
        a[1] = 2.*(K**2 - 1.) / Den
        a[2] = (1. - np.sqrt(2.) * K + K**2) / Den
    else:
        V = 10. ** (Gain / 20.)
        Den = (V + np.sqrt(2. * V) * K + K**2.)
        
        b[0] = V*(1. + np.sqrt(2.) * K + K**2.) / Den
        b[1] = 2. * V*(K**2. - 1.) / Den
        b[2] = V*(1. - np.sqrt(2.) * K + K**2.) / Den;
        
        a[1] = 2. * (K**2. - V) / Den
        a[2] = (V - np.sqrt(2.*V) * K + K**2.) / Den
    
    a[0] = 1.
    return b, a

def makeHighShelf (fc, Gain, fs):
    b = np.zeros (3)
    a = np.zeros (3)

    K = np.tan(np.pi * fc / fs)#bilinear transform frequency correction

    if Gain >= 0:
        V = 10.**(Gain / 20.)
        Den = (1. + np.sqrt(2.) * K + K**2)
        
        b[0] = (V + np.sqrt(2.*V) * K + K**2) / Den
        b[1] = 2. * (K**2 - V) / Den
        b[2] = (V - np.sqrt(2. * V) * K + K**2) / Den
        
        a[1] = 2.*(K**2 - 1.) / Den
        a[2] = (1. - np.sqrt(2.) * K + K**2) / Den
    else:
        V = 10. ** (Gain / 20.)
        Den = (1. + np.sqrt(2. * V) * K + V*K**2.)
        
        b[0] = V*(1. + np.sqrt(2.) * K + K**2.) / Den
        b[1] = 2. * V*(K**2. - 1.) / Den
        b[2] = V*(1. - np.sqrt(2.) * K + K**2.) / Den;
        
        a[1] = 2. * (V*K**2. - 1.) / Den
        a[2] = (1. - np.sqrt(2.*V) * K + V*K**2.) / Den
    
    a[0] = 1.
    return b, a
def makePeakingFilter(fc, Gain, Q, fs):
    K = np.tan(np.pi * fc / fs)#bilinear transform frequency correction
    V = 10.**(Gain / 20.)

    b = np.zeros (3)
    a = np.zeros (3)
    
    if Gain >= 0:
        Den = (1. + 1./Q*K + K**2)
        
        b[0] = (1. + V/Q*K + K**2) / Den
        b[1] = 2.*(K**2 - 1.) / Den     
        b[2] = (1. - V/Q*K + K**2) / Den        
        
        a[0] = 1.
        a[1] = 2.*(K**2 - 1.) / Den
        a[2] = (1. - 1/Q*K + K**2) / Den
    else:
        Den = (1. + 1./(V*Q)*K + K**2)
        
        b[0] = (1. + 1/Q*K + K**2) / Den
        b[1] = 2.*(K**2 - 1.) / Den     
        b[2] = (1. - 1/Q*K + K**2) / Den        
        
        a[0] = 1.
        a[1] = 2.*(K**2 - 1.) / Den
        a[2] = (1. - 1/(V*Q)*K + K**2) / Den
    return b,a

In [34]:
# Example usage of the filter design functions
Fs = 44.1e3 # Sample rate
Fc = 1e3 # cut off frequency
Gain = -10 # yes, it's what it says it is...
Q = 2 # Quality factor

# b, a = makeHighShelf(Fc, 12, Fs)
b, a = makePeakingFilter(Fc, Gain, 2, Fs)

w, h = sc.freqz(b, a)
plt.semilogx(w/np.pi*Fs/2, 20.*np.log10(np.abs(h) + .0000001))
plt.xlim([100, Fs/2])
plt.xlabel('f [Hz]')
plt.ylabel('|H(f)|')